# Install dependencies

In [ ]:
pip install sentence-transformers qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Usi

# Import necessary libraries

In [ ]:
import logging
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Set up logging

In [ ]:
logging.basicConfig(level=logging.INFO)

# Initialize Qdrant Client

In [ ]:
def get_qdrant_client_object(api_key: str, qdrant_url: str) -> QdrantClient:
    """
    Create and return a QdrantClient object for interacting with a Qdrant database.
    """
    return QdrantClient(url=qdrant_url, api_key=api_key)

# Create Qdrant Collection

In [ ]:
def create_qdrant_collection_if_not_exist(qdrant_client: QdrantClient, collection_name: str, vector_size: int) -> QdrantClient:
    """
    Create a Qdrant collection if it does not already exist.
    """
    if not qdrant_client.collection_exists(collection_name):
        qdrant_client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
        )
        logging.info(f"Collection '{collection_name}' created.")
    else:
        logging.info(f"Collection '{collection_name}' already exists.")
    return qdrant_client

# Initialize SBERT Model

In [ ]:
def create_sbert_model(model_name: str = 'all-MiniLM-L6-v2') -> SentenceTransformer:
    """
    Create a SBERT (Sentence-BERT) model for encoding.
    """
    return SentenceTransformer(model_name)

# Create Embeddings

In [ ]:
def create_embedding_for_document(document: str, embedding_model: SentenceTransformer) -> np.array:
    """
    Create an embedding for a given document using an embedding model.
    """
    return embedding_model.encode(document)


# Add Embedding to Qdrant

In [ ]:
def add_embedding_to_qdrant(qdrant_client: QdrantClient, collection_name: str, embedding_vector: list, document: str, doc_id: int) -> None:
    """
    Add an embedding vector and corresponding document information to a Qdrant collection.
    """
    try:
        qdrant_client.upsert(
            collection_name=collection_name,
            points=[PointStruct(
                id=doc_id,
                vector=embedding_vector,
                payload={"text": document}
            )]
        )
        logging.info(f"Document ID {doc_id} added to collection '{collection_name}'.")
    except Exception as e:
        logging.error(f"Failed to add document ID {doc_id} to collection '{collection_name}': {e}")

# Get Query Vector

In [ ]:
def get_query_vector_document(query: str, embedder: SentenceTransformer) -> list:
    """
    Convert a query into a vector using the embedding model.
    """
    return embedder.encode(query).tolist()

# Search Similar Vectors

In [ ]:
def search_similar_vectors(qdrant_client: QdrantClient, collection_name: str, query_vector: list, limit: int = 5):
    """
    Search for vectors in Qdrant that are similar to the query vector.
    """
    return qdrant_client.search(
        collection_name=collection_name, query_vector=query_vector, limit=limit
    )

# Retrieve Text by IDs

In [ ]:
def retrieve_text_by_ids(qdrant_client: QdrantClient, collection_name: str, ids: list) -> list:
    """
    Retrieve the text of documents from Qdrant given their IDs.
    """
    results = qdrant_client.retrieve(collection_name=collection_name, ids=ids)
    return [result.payload["text"] for result in results]

# Main Function

In [ ]:
def main():
    api_key="Br2bpgSJtoNigbCV0W5qHvH6TmucD1ePVsRjZSfvfZ-OnwKNeBPmhg"
    qdrant_url ="https://6747d477-916b-4cf8-b401-efe1302122b7.us-east4-0.gcp.cloud.qdrant.io:6333"
    collection_name = "sbert_collection"

    # Initialize Qdrant client
    qdrant_client = get_qdrant_client_object(api_key, qdrant_url)

    # Initialize SBERT model
    embedder = create_sbert_model('all-MiniLM-L6-v2')
    vector_size = embedder.get_sentence_embedding_dimension()

    # Create Qdrant collection with the correct vector size
    create_qdrant_collection_if_not_exist(qdrant_client, collection_name, vector_size)

    # Define and embed documents
    documents = [
        "Retrieval-Augmented Generation (RAG) combines retrieval and generation for NLP tasks.",
        "RAG uses a retriever to fetch relevant documents and a generator to produce answers.",
        "BERT is a model designed for understanding the context of a word in a sentence.",
        "Generative Pre-trained Transformer (GPT) is a language generation model."
    ]

    vectors = [create_embedding_for_document(doc, embedder) for doc in documents]

    # Add vectors to Qdrant
    for i, (vector, doc) in enumerate(zip(vectors, documents)):
        add_embedding_to_qdrant(qdrant_client, collection_name, vector, doc, i)

    # Example query and retrieval
    query = "Explain RAG in NLP."
    query_vector = get_query_vector_document(query, embedder)
    similar_vectors = search_similar_vectors(qdrant_client, collection_name, query_vector)
    similar_ids = [result.id for result in similar_vectors]
    similar_texts = retrieve_text_by_ids(qdrant_client, collection_name, similar_ids)

    # Display results
    print(similar_texts)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

['RAG uses a retriever to fetch relevant documents and a generator to produce answers.', 'Retrieval-Augmented Generation (RAG) combines retrieval and generation for NLP tasks.', 'BERT is a model designed for understanding the context of a word in a sentence.', 'Generative Pre-trained Transformer (GPT) is a language generation model.']
